<div align="center">

# Exam Project

## Programming, Algorithms and Data Structures [KAN-CDSCO2402U]

**Tobias Madsen**



*MSc in Business Administration and Data Science*

</div>

# Project Overview

This notebook demonstrates the **functionality and tests** of the classes created for my project. The primary purpose of this notebook is to ensure that all the implemented classes work correctly by running unit tests. These classes include:

- **Users** (`User`, `RegularUser`, `StudentUser`, `AdminUser`)  
- **Inventory**  
- **Sandwich**  
- **Loyalty Program**  
- **Order**

Each class has been thoroughly tested with relevant test cases to validate its functionality.

---

## Running the Notebook

Running this notebook will execute the tests to ensure the classes pass all assertions. It does not demonstrate the application in action but confirms that the building blocks (classes) are working as intended.

---

## Application Details

The classes defined in this project are located in the `helper_functions/classes.py` file and are integrated into the **Streamlit app** (`app.py`). To experience the complete functionality of the project (e.g., user interface, order management, analytics, etc.), you can access the application through the following link:

## [❗️ TO RUN THE APP, GO TO THIS WEBSITE🧑‍💻🥪](https://prog-exam-cbs.streamlit.app)
*Note: This link is safe to use and redirects to the app hosted on Streamlit's cloud platform. Streamlit ensures secure hosting and reliable access to the application. - Here's the link for transparency: https://prog-exam-cbs.streamlit.app*


# Classes

## Users

### Base Class


In [1]:
class User:
    def __init__(self, user_id, name, email, phone):
        """
        Initialize a new User object with the parameters: user_id, name, email, and phone.
        """
        self.user_id = user_id
        self.name = name
        self.email = email
        self.phone = phone

    def update_contact_info(self, email=None, phone=None):
        """
        Update the user's contact information with a new email and/or phone number (both are optional).
        """
        if email:
            self.email = email
        if phone:
            self.phone = phone

    def apply_discount(self, total_cost):
        """
        Default: no discount for a regular user.
        Override in subclasses for specific discount logic.
        """
        return total_cost  # Default: no discount

    def __str__(self):
        """
        Formatted string representation of the user.
        """
        return f"User ID: {self.user_id}\nName: {self.name}\nEmail: {self.email}\nPhone: {self.phone}"

#### Subclass: Regular User

In [2]:
class RegularUser(User):
    def __init__(self, user_id, name, email, phone):
        super().__init__(user_id, name, email, phone)
        self.order_history = []
        self.sandwich_count = 0

    def add_order(self, order):
        """
        Add an order to the user's order history and update sandwich count.
        """
        self.order_history.append(order)
        self.sandwich_count += len(order.sandwiches)

    def get_order_history(self):
        """
        Retrieve and return the user's order history.
        """
        return self.order_history

    def __str__(self):
        return super().__str__() + f"\nTotal Sandwiches Purchased: {self.sandwich_count}"

#### Subclass: Student User


In [3]:
class StudentUser(RegularUser):
    def __init__(self, user_id, name, email, phone, domain="@student.cbs.dk", discount_rate=0.05):
        super().__init__(user_id, name, email, phone)
        self.student_domain = domain
        self.student_discount_rate = discount_rate

    def apply_discount(self, total_cost):
        """
        Apply a 5% discount if the user's email ends with the student domain.
        """
        if self.email.lower().endswith(self.student_domain):
            return total_cost * (1 - self.student_discount_rate)
        return total_cost

    def __str__(self):
        return super().__str__() + "\nStatus: Student"

#### Subclass: Admin User


In [4]:
class AdminUser(User):
    def __init__(self, user_id, name, email, phone):
        super().__init__(user_id, name, email, phone)

    def view_all_orders(self, orders):
        """
        View all orders in the system (admin-only feature).
        """
        if not orders:
            return "No orders available."
        return orders
    
    def view_all_customers(self, customers):
        """
        View all customers in the system. If no customers exist, return an empty list.
        """
        return customers or []

    def manage_inventory(self, inventory, action, category, item_name, price=None):
        """
        Manage the inventory by adding or removing items (admin-only feature).
        """
        if action == "add":
            inventory.add_ingredient(category, item_name, price)
        elif action == "remove":
            inventory.remove_ingredient(category, item_name)
        else:
            raise ValueError("Invalid action. Use 'add' or 'remove'.")

    def __str__(self):
        return super().__str__() + "\nRole: Admin"

## Inventory

In [5]:
class Inventory:
    """
    Manages available ingredients and their corresponding prices.
    Allows adding and removing ingredients.
    """
    def __init__(self):
        # Initialize with some default values
        self.available_breads = {
            "White": 0,
            "Whole Wheat": 0
        }
        self.available_spreads = {
            "No spread": 0,
            "Chilimayo": 0,
            "Plain cream cheese": 0,
            "Hummus": 0
        }
        self.available_proteins = {
            "No protein": 0,
            "Chorizo": 0,
            "Chicken": 0,
            "Tandoori chicken": 0,
            "Tuna": 0,
            "Turkey": 0
        }
        self.available_vegetables = {
            "No vegetables": 0,
            "Iceberg": 0,
            "Mixed salad": 0,
            "Corn": 0,
            "Red onion": 0,
            "Feta": 0,
            "Jalapeños": 0,
            "Sundried tomatoes": 0,
            "Bell pepper": 0,
            "Carrot": 0,
            "Pickles": 0,
            "Tomato": 0,
            "Cucumber": 0,
            "Olive": 0
        }
        self.available_extras = {
            "No extras": 0,
            "Avocado": 6,
            "Cheddar cheese": 6,
            "Turkey bacon": 6
        }
        self.available_dressings = {
            "No Dressing": 0,
            "Sour cream dressing": 0,
            "Curry dressing": 0,
            "Pesto": 0,
            "Sweet chili dressing": 0,
            "Strong chili dressing": 0
        }

    def add_ingredient(self, category, name, price=0):
        """
        Add a new ingredient to a given category with a specified price.
        category: str, one of ["bread", "spread", "protein", "vegetable", "extra", "dressing"]
        name: str, name of the ingredient
        price: int or float, optional price of the ingredient (for extras or special breads)
        """
        category_dict = self._get_category_dict(category)
        if name in category_dict:
            raise ValueError(f"Ingredient '{name}' already exists.")
        category_dict[name] = price

    def remove_ingredient(self, category, name):
        """
        Remove an ingredient from a given category.
        """
        category_dict = self._get_category_dict(category)
        if name not in category_dict:
            raise ValueError(f"Ingredient '{name}' does not exist in {category} category.")
        if name.startswith("No "):      # Prevent removal of "No ..." options if you consider them mandatory placeholders
            raise ValueError(f"Cannot remove mandatory ingredient '{name}'.")
        del category_dict[name]

    def _get_category_dict(self, category):
        category_mapping = {
            "bread": self.available_breads,
            "spread": self.available_spreads,
            "protein": self.available_proteins,
            "vegetable": self.available_vegetables,
            "extra": self.available_extras,
            "dressing": self.available_dressings,
        }
        return category_mapping.get(category.lower())

    def is_valid_bread(self, bread):
        return bread in self.available_breads

    def is_valid_spread(self, spread):
        return spread in self.available_spreads

    def is_valid_protein(self, protein):
        return protein in self.available_proteins

    def is_valid_vegetables(self, vegetables):
        return all(veg in self.available_vegetables for veg in vegetables)

    def is_valid_dressing(self, dressing):
        return dressing in self.available_dressings

    def is_valid_extras(self, extras):
        return all(extra in self.available_extras for extra in extras)

    def get_extra_cost(self, extras):
        return sum(self.available_extras.get(e, 0) for e in extras if e != "No extras")

## Sandwich

In [6]:
class Sandwich:
    def __init__(self, inventory=None):
        """
        Initialize a new Sandwich object.
        Parameters:
        - inventory: An Inventory object to validate ingredient selections. Defaults to a new Inventory instance.
        """
        self.inventory = inventory if inventory else Inventory()
        self.bread = None
        self.spread = None
        self.protein = None
        self.vegetables = []
        self.dressing = None
        self.extras = []

    def get_price(self, base_price):
        """
        Calculates the price of the sandwich based on:
        - Base price
        - Cost of extra ingredients
        """
        valid_extras = [extra for extra in self.extras if extra != "No extras"]
        extra_cost = self.inventory.get_extra_cost(valid_extras)
        return base_price + extra_cost

    def select_bread(self, bread):
        if self.inventory.is_valid_bread(bread):
            self.bread = bread
        else:
            raise ValueError("Invalid bread choice.")

    def select_spread(self, spread):
        if self.inventory.is_valid_spread(spread):
            self.spread = spread
        else:
            raise ValueError("Invalid spread choice.")

    def select_protein(self, protein):
        if self.inventory.is_valid_protein(protein):
            self.protein = protein
        else:
            raise ValueError("Invalid protein choice.")

    def add_vegetables(self, vegetables):
        if self.inventory.is_valid_vegetables(vegetables):
            self.vegetables = vegetables
        else:
            raise ValueError("One or more invalid vegetable choices.")

    def select_dressing(self, dressing):
        if self.inventory.is_valid_dressing(dressing):
            self.dressing = dressing
        else:
            raise ValueError("Invalid dressing choice.")

    def add_extras(self, extras):
        if self.inventory.is_valid_extras(extras):
            self.extras = extras
        else:
            raise ValueError("One or more invalid extras.")

    def __str__(self):
        return (
            f"Bread: {self.bread}\n"
            f"Spread: {self.spread}\n"
            f"Protein: {self.protein}\n"
            f"Vegetables: {', '.join(self.vegetables)}\n"
            f"Dressing: {self.dressing}\n"
            f"Extras: {', '.join(self.extras)}"
        )

## Loyalty Program

In [7]:
class Loyalty:
    """
    Encapsulates the logic for loyalty benefits:
    - Every nth sandwich (based on threshold) is free.
    """
    def __init__(self, threshold=10):
        self.threshold = threshold  # e.g. every 10th sandwich is free

    def free_sandwiches_earned(self, previous_sandwich_count, current_sandwich_count):
        """
        Given the customer's previous sandwich count and the new cumulative count (after current order),
        determine how many free sandwiches should be awarded in the current order.
        """
        total_free_sandwiches_after = current_sandwich_count // self.threshold
        total_free_sandwiches_before = previous_sandwich_count // self.threshold

        # Difference is how many free sandwiches to apply to this order
        return total_free_sandwiches_after - total_free_sandwiches_before

## Order

In [8]:
from datetime import datetime

class Order:
    def __init__(self, order_id, customer, order_time=None, inventory=None, loyalty_program=None):
        """
        Initializes an Order instance with order details and dependencies.
        """
        self.order_id = order_id
        self.customer = customer
        self.sandwiches = []
        self.status = "Pending"
        self.order_time = order_time if order_time else datetime.now()
        self.inventory = inventory if inventory else Inventory()
        self.loyalty_program = loyalty_program if loyalty_program else Loyalty(10)

    def add_sandwich(self, sandwich):
        """
        Adds a sandwich to the order.
        """
        if isinstance(sandwich, Sandwich):
            self.sandwiches.append(sandwich)
        else:
            raise ValueError("Only Sandwich objects can be added.")

    def calculate_total(self):
        base_price = self.get_time_based_price()
        sandwich_costs = [s.get_price(base_price) for s in self.sandwiches]

        free_sandwich_count = self.loyalty_program.free_sandwiches_earned(
            self.customer.sandwich_count, self.customer.sandwich_count + len(self.sandwiches)
        )

        # Apply loyalty discount
        sandwich_costs.sort()
        for _ in range(free_sandwich_count):
            if sandwich_costs:
                sandwich_costs.pop(0)

        total = sum(sandwich_costs)
        total_with_discount = self.customer.apply_discount(total)

        st_discount = f"\nLoyalty Program: {free_sandwich_count} Sandwich(es) Free" if free_sandwich_count else ""
        return total_with_discount, st_discount

    def get_time_based_price(self):
        """
        Returns the base price of a sandwich based on the time of the order.
        Price is 77 DKK during 8:00 - 14:00 and 80 DKK at other times.
        """
        hour = self.order_time.hour
        return 77 if 8 <= hour < 14 else 80

    def update_status(self, new_status):
        """
        Updates the status of the order.
        """
        self.status = new_status

    def __str__(self):
        if not self.sandwiches:
            return "No sandwiches in this order."

        sandwiches_str = "\n\n".join(
            [f"Sandwich {i+1}:\n{str(s)}" for i, s in enumerate(self.sandwiches)]
        )
        total, st_discount = self.calculate_total()

        return (f"Order ID: {self.order_id}\n"
                f"Customer: {self.customer.name} (ID: {self.customer.user_id})\n"
                f"Sandwiches:\n{sandwiches_str}\n"
                f"Status: {self.status}\n"
                f"Total Cost: {total:.2f} DKK{st_discount}")


# Tests

## User Test

In [9]:
def test_users():
    # Regular User
    regular_user = RegularUser("001", "Alice", "alice@example.com", "1234567890")
    assert regular_user.name == "Alice"
    assert regular_user.sandwich_count == 0
    print("RegularUser test passed")

    # Student User
    student_user = StudentUser("002", "Bob", "bob@student.cbs.dk", "9876543210")
    assert student_user.apply_discount(100) == 95  # 5% discount
    assert student_user.apply_discount(200) == 190
    assert student_user.email.endswith("@student.cbs.dk")
    print("StudentUser discount test passed")

    # Admin User
    admin_user = AdminUser("003", "Admin", "admin@example.com", "5551234567")
    assert admin_user.view_all_orders([]) == "No orders available."
    assert admin_user.view_all_customers([]) == []
    print("AdminUser tests passed")

## Inventory Test

In [10]:
def test_inventory():
    inventory = Inventory()

    # Add ingredient
    inventory.add_ingredient("bread", "Gluten-Free", 5)
    assert "Gluten-Free" in inventory.available_breads

    # Remove ingredient
    inventory.remove_ingredient("bread", "Gluten-Free")
    assert "Gluten-Free" not in inventory.available_breads

    # Validate ingredients
    assert inventory.is_valid_bread("White")
    assert not inventory.is_valid_bread("FakeBread")
    assert inventory.is_valid_extras(["Avocado", "Cheddar cheese"])
    print("Inventory tests passed")

## Sandwich Test

In [11]:
def test_sandwich():
    inventory = Inventory()
    sandwich = Sandwich(inventory)

    # Set attributes
    sandwich.select_bread("White")
    sandwich.select_spread("Hummus")
    sandwich.select_protein("Chicken")
    sandwich.add_vegetables(["Tomato", "Cucumber"])
    sandwich.select_dressing("Pesto")
    sandwich.add_extras(["Avocado", "Cheddar cheese"])

    assert sandwich.bread == "White"
    assert sandwich.spread == "Hummus"
    assert sandwich.protein == "Chicken"
    assert "Avocado" in sandwich.extras

    # Calculate price
    base_price = 77
    price = sandwich.get_price(base_price)  # No need to pass inventory here
    assert price == base_price + 12  # Avocado (6) + Cheddar cheese (6)
    print("Sandwich tests passed")




## Loyalty Test

In [12]:
def test_loyalty():
    loyalty = Loyalty(threshold=10)

    # No free sandwiches yet
    free_sandwiches = loyalty.free_sandwiches_earned(5, 9)
    assert free_sandwiches == 0

    # Cross the threshold
    free_sandwiches = loyalty.free_sandwiches_earned(9, 10)
    assert free_sandwiches == 1

    # Multiple thresholds
    free_sandwiches = loyalty.free_sandwiches_earned(15, 25)
    assert free_sandwiches == 1  # Only one new threshold crossed
    print("Loyalty tests passed")

## Order Test

In [13]:
from datetime import datetime, timedelta

def test_order():
    inventory = Inventory()
    customer = RegularUser("001", "Alice", "alice@example.com", "1234567890")
    order_time = datetime.now()
    loyalty = Loyalty()

    # Create order
    order = Order(order_id=1, customer=customer, order_time=order_time, inventory=inventory, loyalty_program=loyalty)

    # Add sandwiches
    sandwich1 = Sandwich(inventory)
    sandwich1.select_bread("White")
    sandwich1.add_extras(["Avocado"])
    order.add_sandwich(sandwich1)

    sandwich2 = Sandwich(inventory)
    sandwich2.select_bread("Whole Wheat")
    order.add_sandwich(sandwich2)

    assert len(order.sandwiches) == 2

    # Calculate total
    total, _ = order.calculate_total()
    base_price = order.get_time_based_price()
    expected_price = base_price * 2 + 6  # Two sandwiches, one with Avocado
    assert total == expected_price
    print("Order tests passed")

In [14]:
def run_tests():
    print("Running User tests...")
    test_users()
    
    print("\nRunning Inventory tests...")
    test_inventory()
    
    print("\nRunning Sandwich tests...")
    test_sandwich()
    
    print("\nRunning Loyalty tests...")
    test_loyalty()
    
    print("\nRunning Order tests...")
    test_order()
    
    print("\nAll tests passed!")

## → Run all tests

In [15]:
run_tests()

Running User tests...
RegularUser test passed
StudentUser discount test passed
AdminUser tests passed

Running Inventory tests...
Inventory tests passed

Running Sandwich tests...
Sandwich tests passed

Running Loyalty tests...
Loyalty tests passed

Running Order tests...
Order tests passed

All tests passed!
